<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Find S3 buckets without any lifecycle policies and attach one to them</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-RDS-Instances-with-Low-CPU-Utilization"><u>Add Lifecycle Policy To AWS S3 Buckets</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find S3 Buckets without Lifecycle Policies</a><br>2)<a href="#2" target="_self" rel="noopener"> Attach Lifecycle Policy</a></p>

In [17]:
if region == None:
    region = ''
if bucket_names and not region:
    raise SystemExit("Provide a region for the S3 Bucket names!")

<h3 id="Find-RDS-Instances-with-Low-CPU-Utilization"><a id="1" target="_self" rel="nofollow"></a>Find S3 Buckets without Lifecycle Policies</h3>
<p>Using Find AWS S3 buckets without lifecycle policieswe can identify buckets that do not have any configured lifecycle rules for managing object lifecycle. By examining the presence or absence of lifecycle policies, you can gain insights into the data management practices of your S3 buckets. This information can be valuable for optimizing storage costs and ensuring efficient data lifecycle management.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>buckets_without_policy</code></p>
</blockquote>

In [18]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.aws.aws_get_s3_buckets.aws_get_s3_buckets import aws_get_s3_buckets
from typing import List, Optional, Tuple
import pprint


from beartype import beartype
@beartype
def aws_find_s3_buckets_without_lifecycle_policies_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_find_s3_buckets_without_lifecycle_policies(handle, region: str="") -> Tuple:
    """aws_find_s3_buckets_without_lifecycle_policies List all the S3 buckets without lifecycle policies

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: AWS Region of the bucket

        :rtype: Status, List of all the S3 buckets without lifecycle policies with regions
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            s3Session = handle.resource("s3", region_name=reg)
            response = aws_get_s3_buckets(handle, region=reg)
            for bucket in response:
                bucket_lifecycle_configuration = s3Session.BucketLifecycleConfiguration(bucket)
                try:
                    if bucket_lifecycle_configuration.rules:
                        continue
                except Exception:
                    bucket_details = {}
                    bucket_details["bucket_name"] = bucket
                    bucket_details["region"] = reg
                    result.append(bucket_details)
        except Exception:
            pass
    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not bucket_names",
    "condition_result": true
    }''')
task.configure(outputName="buckets_without_policy")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_find_s3_buckets_without_lifecycle_policies, lego_printer=aws_find_s3_buckets_without_lifecycle_policies_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Low-CPU-Utilization-RDS-Instances">Create List of Buckets with No Lifecycle Policy</h3>
<p>This action gets the list of&nbsp; S3 buckets from the tuple output in Step 1.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_buckets_without_policy</code></p>
</blockquote>

In [19]:
all_buckets_without_policy = []
try:
    for res in buckets_without_policy:
        if type(res)==bool:
            if res == False:
                continue
        elif type(res)==list:
            if len(res)!=0:
                all_buckets_without_policy=res
except Exception:
    for buck in bucket_names:
        data_dict = {}
        data_dict["region"] = region
        data_dict["bucket_name"] = buck
        all_buckets_without_policy.append(data_dict)
print(all_buckets_without_policy)
task.configure(outputName="all_buckets_without_policy")

<h3 id="Delete-RDS-Instance"><a id="2" target="_self" rel="nofollow"></a>Attach Lifecycle Policy</h3>
<p>This action attached a new lifecycle policy to the S3 buckets found in Step 1. From the listed input parameters, <code>expiration_days and nonconcurrent_days</code> have a default value of 30 days.</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>bucket_name, region, expiration_days, prefix, nonconcurrent_days</code></p>
</blockquote>

In [21]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict, Optional
import pprint


from beartype import beartype
@beartype
def aws_add_lifecycle_configuration_to_s3_bucket_printer(output):
    if output is None:
        return
    pprint.pprint(output)



@beartype
def aws_add_lifecycle_configuration_to_s3_bucket(handle, region: str, bucket_name:str, expiration_days:int=30, prefix:str='', noncurrent_days:int=30) -> Dict:
    """aws_add_lifecycle_configuration_to_s3_bucket returns response of adding lifecycle configuration

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: location of the bucket

        :type bucket_name: string
        :param bucket_name: The name of the bucket for which to set the configuration.

        :type expiration_days: int
        :param expiration_days: Specifies the expiration for the lifecycle of the object in the form of days. Eg: 30 (days)

        :type prefix: string
        :param prefix: location of the bucket

        :type noncurrent_days: int
        :param noncurrent_days: Specifies the number of days an object is noncurrent before Amazon S3 permanently deletes the noncurrent object versions.

        :rtype: Dict of the response of adding lifecycle configuration
    """
    s3Client = handle.client("s3", region_name=region)
    try:
        lifecycle_config = {
            'Rules': [
                {
                    'Expiration': {
                        'Days': expiration_days,
                    },
                    'Filter': {
                        'Prefix': ''
                    },
                    'Status': 'Enabled',
                    'NoncurrentVersionExpiration': {
                        'NoncurrentDays': noncurrent_days
                    }
                }
            ]
        }
        bucket_name = 'testrunbook'
        response = s3Client.put_bucket_lifecycle_configuration(
            Bucket=bucket_name,
            LifecycleConfiguration=lifecycle_config
        )
    except Exception as e:
        raise e
    return response




task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "expiration_days": "int(expiration_days)",
    "prefix": "prefix",
    "noncurrent_days": "int(noncurrent_days)",
    "bucket_name": "iter.get(\\"bucket_name\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_buckets_without_policy",
    "iter_parameter": ["bucket_name","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_buckets_without_policy)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_add_lifecycle_configuration_to_s3_bucket, lego_printer=aws_add_lifecycle_configuration_to_s3_bucket_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able to find AWS S3 buckets without lifecycle policies and attach one to them. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>